In [17]:
import PRF
from sklearn.ensemble import RandomForestClassifier
import numpy
from sklearn.impute import SimpleImputer as Imputer # for new versions for sklearn

In [3]:
X = numpy.load('data/bootstrap_X.npy')
y = numpy.load('data/bootstrap_y.npy')
print(X)
print(y)
print(numpy.mean(y))
print(numpy.std(y))
y[y > 2] = 2
print(y)

n_objects = X.shape[0]
n_features = X.shape[1]
print(n_objects, 'objects,', n_features, 'features')

shuffled_inds = numpy.random.choice(numpy.arange(n_objects),n_objects,replace=False)
print("Shuffled Indices: ", shuffled_inds)
print(len(shuffled_inds))
shuffled_inds = numpy.where( (y == 1)  |  (y == 2) |  (y == 4)|  (y == 5)|  (y == 6)|  (y == 8)|  (y == 13))[0]
print("Y Cut Shuffled Indices: ", shuffled_inds)
print(len(shuffled_inds))
shuffled_inds = numpy.random.choice(shuffled_inds,len(shuffled_inds),replace=False)
print("Re-Shuffled Indices? ", shuffled_inds)
n_train = 10000
n_test = 1000
print('Train set size = {}, Test set size = {}'.format(n_train, n_test))

nf = n_features
train_inds = shuffled_inds[:n_train]
X_train = X[train_inds][:,:nf]
y_train = y[train_inds]

test_inds = shuffled_inds[n_train:(n_train + n_test)]
X_test = X[test_inds][:,:nf]
y_test = y[test_inds]

[[ 0.33453338  0.3817734   0.19323093 ... -0.81191589  0.19067206
   0.84772759]
 [ 0.32742773  0.35012285  0.17785153 ... -0.85643206  0.19620735
   0.85023905]
 [ 0.39173798  0.32019704  0.17513842 ...  0.23618577  0.20204299
   0.79843681]
 ...
 [ 0.403697    0.27038462  0.12964029 ...  0.89465122  0.1941173
   0.77770493]
 [ 0.25029907  0.33824084  0.20026945 ...  0.8638979   0.14645717
   0.82934586]
 [ 0.27610644  0.34        0.16186233 ... -0.52845195  0.15704813
   0.83015366]]
[5 1 4 ... 1 5 1]
2.34630222977833
2.584966902039068
[2 1 2 ... 1 2 1]
45879 objects, 17 features
Shuffled Indices:  [ 6690 21266  7081 ... 13535 25108 44330]
45879
Y Cut Shuffled Indices:  [    0     1     2 ... 45876 45877 45878]
45879
Re-Shuffled Indices?  [ 6965 11798  6789 ... 43515  5566 11678]
Train set size = 10000, Test set size = 1000


In [4]:
n_trees = 100
prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True)
prf_cls.fit(X=X_train, y=y_train)
prf_cls.score(X_test, y=y_test)

0.823

In [7]:
from eli5.permutation_importance import get_score_importances

# ... load data, define score function
def score(X, y):
    y_pred = predict(X)
    return accuracy(y, y_pred)

base_score, score_decreases = get_score_importances(prf_cls.score, X_test, y_test)
print(base_score, score_decreases)
feature_importances = numpy.mean(score_decreases, axis=0)
feature_importances

0.823 [array([ 0.   ,  0.005,  0.008, -0.006,  0.005,  0.002,  0.006,  0.006,
        0.001,  0.026,  0.016, -0.002,  0.009,  0.107,  0.01 ,  0.011,
        0.007]), array([-0.002,  0.009,  0.009, -0.002,  0.003,  0.001,  0.014,  0.01 ,
        0.008,  0.028,  0.022,  0.   ,  0.015,  0.116,  0.007,  0.008,
        0.003]), array([ 0.006,  0.009,  0.003, -0.001,  0.012,  0.004,  0.008,  0.01 ,
        0.01 ,  0.024,  0.022, -0.003,  0.003,  0.1  ,  0.007,  0.014,
        0.   ]), array([ 0.002,  0.011,  0.012, -0.003,  0.003,  0.003,  0.014,  0.008,
        0.007,  0.031,  0.027,  0.002,  0.004,  0.084,  0.007,  0.009,
        0.002]), array([-0.002,  0.009,  0.006, -0.001,  0.009,  0.002,  0.014,  0.012,
        0.008,  0.035,  0.022, -0.002,  0.009,  0.09 ,  0.012,  0.011,
        0.001])]


array([ 0.0008,  0.0086,  0.0076, -0.0026,  0.0064,  0.0024,  0.0112,
        0.0092,  0.0068,  0.0288,  0.0218, -0.001 ,  0.008 ,  0.0994,
        0.0086,  0.0106,  0.0026])

In [20]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel

# ... load data
imp = Imputer(missing_values=numpy.nan, strategy='median')
X_train_w_nans_imp = imp.fit_transform(X_train)
X,y = X_train_w_nans_imp, y_train

perm = PermutationImportance(SVC(), cv=5)
perm.fit(X, y)

# perm.feature_importances_ attribute is now available, it can be used
# for feature selection - let's e.g. select features which increase
# accuracy by at least 0.05:
sel = SelectFromModel(perm, threshold=0.05, prefit=True)
X_trans = sel.transform(X)

# It is possible to combine SelectFromModel and
# PermutationImportance directly, without fitting
# PermutationImportance first:
sel = SelectFromModel(
    PermutationImportance(SVC(), cv=5),
    threshold=0.05,
).fit(X, y)
X_trans = sel.transform(X)

/home/aaron/anaconda3/envs/astroconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/aaron/anaconda3/envs/astroconda/lib/python3.6/site-packages/sklearn/feature_selection/_base.py:81: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)
/home/aaron/anaconda3/envs/astroconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/aaron/anaconda3/envs/astroconda/lib/python3.6/site-packages/sklearn/feature_selection/_base.py:81: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)


In [21]:
X_trans

array([], shape=(20000, 0), dtype=float64)

In [8]:
import numpy
import pandas as pd
data = pd.read_csv('data/cflvsc_r01_crosssources.dat',na_values=-999)
data = data[data[' cflvsc.MainVarType'] != '  Others']
print(data.columns)

data['zeroes'] = numpy.zeros(len(data[' cflvsc.Avar']))

data.loc[data[' cflvsc.Ngoodmeasures'] < 40,' cflvsc.MainVarType'] = 'Spurious'
data[' cflvsc.MainVarType'].value_counts()

Index(['#cflvsc.sourceID', ' cflvsc.ra(J2000)', ' cflvsc.dec(J2000)',
       ' cflvsc.gl(J2000)', ' cflvsc.gb(J2000)', ' cflvsc.zAperMag3',
       ' cflvsc.zAperMag3Err', ' cflvsc.yAperMag3', ' cflvsc.yAperMag3Err',
       ' cflvsc.jAperMag3', ' cflvsc.jAperMag3Err', ' cflvsc.hAperMag3',
       ' cflvsc.hAperMag3Err', ' cflvsc.ksAperMag3', ' cflvsc.ksAperMag3Errs',
       ' cflvsc.ksEMeanMagPawprint', ' cflvsc.ED', ' cflvsc.ExpRMS_Noise',
       ' cflvsc.Ngoodmeasures', ' cflvsc.Xindex', ' cflvsc.Kfi2', ' cflvsc.L2',
       ' cflvsc.Ncorrelation2', ' cflvsc.FAPcorrelation2',
       ' cflvsc.FlagDataType', ' cflvsc.ebv', ' cflvsc.ebverr',
       ' cflvsc.FreqKfi2', ' cflvsc.HeightKfi2toKfi2', ' cflvsc.HeightKfi2',
       ' cflvsc.FreqLfl2', ' cflvsc.HeightKfi2toLfl2', ' cflvsc.HeightLfl2',
       ' cflvsc.FreqLSG', ' cflvsc.HeightKfi2toLSG', ' cflvsc.HeightLSG',
       ' cflvsc.FreqPDM', ' cflvsc.HeightKfi2toPDM', ' cflvsc.HeightPDM',
       ' cflvsc.FreqSTR', ' cflvsc.HeightKfi2toSTR',

  EB        109479
Spurious     28017
  RR         27383
  NSIN       11748
  YSO         5757
             ...  
  BLAP           1
  AM             1
  UGSU           1
  EXOR           1
  AHB            1
Name:  cflvsc.MainVarType, Length: 63, dtype: int64

In [12]:
#Remove columns with too few variables in them

counts = data[' cflvsc.MainVarType'].value_counts()
valid = counts[counts > 20]
print(valid)
data = data[data[' cflvsc.MainVarType'].isin(valid.index.values)]

for inverse_column in [' cflvsc.HeightKfi2',' cflvsc.HeightLfl2',' cflvsc.HeightLSG',' cflvsc.HeightPDM',
        ' cflvsc.HeightSTR']:
    columnName = inverse_column.replace(" cflvsc.",'Inverse')
    data[columnName] = numpy.ones(len(data[inverse_column]))
    data[columnName] = data[columnName].div(data[inverse_column])
    
    
X=data[[' cflvsc.zAperMag3',
       ' cflvsc.zAperMag3Err', ' cflvsc.yAperMag3', ' cflvsc.yAperMag3Err',
       ' cflvsc.jAperMag3', ' cflvsc.jAperMag3Err', ' cflvsc.hAperMag3',
       ' cflvsc.hAperMag3Err', ' cflvsc.ksAperMag3', ' cflvsc.ksAperMag3Errs',
       ' cflvsc.ksEMeanMagPawprint', ' cflvsc.ED', ' cflvsc.ExpRMS_Noise',
       ' cflvsc.Ngoodmeasures', ' cflvsc.Xindex', ' cflvsc.Kfi2', ' cflvsc.L2',
       ' cflvsc.Ncorrelation2', ' cflvsc.FAPcorrelation2',
       ' cflvsc.ebv', ' cflvsc.ebverr',
       ' cflvsc.FreqKfi2', ' cflvsc.HeightKfi2',
       ' cflvsc.FreqLfl2', ' cflvsc.HeightLfl2',
       ' cflvsc.FreqLSG', ' cflvsc.HeightLSG',
       ' cflvsc.FreqPDM', ' cflvsc.HeightPDM',
       ' cflvsc.FreqSTR', ' cflvsc.HeightSTR',
       ' cflvsc.Avar']].to_numpy()

#Maybe flag objects with low Ncorrelation with a different MainVarType flag - that was it can identify them as bad

X=data[[' cflvsc.zAperMag3',' cflvsc.yAperMag3',' cflvsc.jAperMag3', ' cflvsc.hAperMag3',
        ' cflvsc.ksAperMag3',
       ' cflvsc.FreqKfi2',' cflvsc.FreqLfl2',' cflvsc.FreqLSG',' cflvsc.FreqPDM',
       ' cflvsc.FreqSTR',
       ' cflvsc.Avar',' cflvsc.Kfi2', ' cflvsc.L2',' cflvsc.ebv'
       ]].to_numpy()

dX = data[[' cflvsc.zAperMag3Err', ' cflvsc.yAperMag3Err',' cflvsc.jAperMag3Err',' cflvsc.hAperMag3Err', 
           ' cflvsc.ksAperMag3Errs',
        'InverseHeightKfi2','InverseHeightLfl2','InverseHeightLSG','InverseHeightPDM',
        'InverseHeightSTR',
           'zeroes','zeroes','zeroes',' cflvsc.ebverr'
          ]].to_numpy()

#dX = data[[' cflvsc.zAperMag3Err', ' cflvsc.yAperMag3Err',' cflvsc.jAperMag3Err',' cflvsc.hAperMag3Err', 
#           ' cflvsc.ksAperMag3Errs',
#        ' cflvsc.HeightKfi2',' cflvsc.HeightLfl2',' cflvsc.HeightLSG',' cflvsc.HeightPDM',
#        ' cflvsc.HeightSTR',
#           'zeroes','zeroes','zeroes',' cflvsc.ebverr','zeroes'
#          ]].to_numpy()

y=data[' cflvsc.MainVarType'].to_numpy()
print(X)
print(y)

n_objects = X.shape[0]
n_features = X.shape[1]
print(n_objects, 'objects,', n_features, 'features')

shuffled_inds = numpy.random.choice(numpy.arange(n_objects),n_objects,replace=False)
print("Shuffled Indices: ", shuffled_inds)
print(len(shuffled_inds))
shuffled_inds = numpy.random.choice(shuffled_inds,len(shuffled_inds),replace=False)
print("Re-Shuffled Indices? ", shuffled_inds)
n_train = 20000
n_test = 2000
print('Train set size = {}, Test set size = {}'.format(n_train, n_test))

nf = n_features
train_inds = shuffled_inds[:n_train]
X_train = X[train_inds][:,:nf]
y_train = y[train_inds]
dX_train = dX[train_inds][:,:nf]

test_inds = shuffled_inds[n_train:(n_train + n_test)]
X_test = X[test_inds][:,:nf]
y_test = y[test_inds]
dX_test = dX[test_inds][:,:nf]

  EB           109479
Spurious        28017
  RR            27383
  NSIN          11748
  YSO            5757
  iC             4208
  E              2842
  FKCOM          1565
  X              1488
  IR             1369
  EW             1179
  EA             1152
  Radio           817
  ISM             800
  V*              756
  PUL             701
  SR              440
  LPV             415
  RGB             320
  Planet          267
  CW              262
  PER             258
  Microlens       196
  TTS             190
  M               156
  DCEP            100
  BE               87
  N                62
  ROT              54
  CEP              46
  WR               35
  CV               33
  DSCT             31
  IN               30
  UG               29
  RV               28
  TTau             27
  APER             26
  GRB              23
Name:  cflvsc.MainVarType, dtype: int64
[[14.76163  14.657574 14.700644 ...  0.804878  1.425342  0.      ]
 [      nan       nan       nan ...

In [13]:
from sklearn.impute import SimpleImputer as Imputer # for new versions for sklearn

n_trees = 100
prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True)
prf_cls.fit(X=X_train, y=y_train, dX = dX_train)
#prf_cls.score(X_test, y=y_test)
print('PRF: {}'.format(prf_cls.score(X_test, y=y_test, dX=dX_test)))

PRF: 0.737


In [14]:
# ... load data, define score function
def score(X, y):
    y_pred = predict(X)
    return accuracy(y, y_pred)

base_score, score_decreases = get_score_importances(prf_cls.score, X_test, y_test)
print(base_score, score_decreases)
feature_importances = numpy.mean(score_decreases, axis=0)
feature_importances

0.738 [array([0.    , 0.    , 0.    , 0.    , 0.    , 0.0045, 0.0055, 0.0605,
       0.015 , 0.0045, 0.169 , 0.011 , 0.0365, 0.0405]), array([0.    , 0.    , 0.    , 0.    , 0.    , 0.0065, 0.01  , 0.0665,
       0.0155, 0.0025, 0.165 , 0.013 , 0.0285, 0.0445]), array([0.    , 0.    , 0.    , 0.    , 0.    , 0.0075, 0.011 , 0.0645,
       0.016 , 0.0015, 0.1615, 0.008 , 0.031 , 0.036 ]), array([0.    , 0.    , 0.    , 0.    , 0.    , 0.0085, 0.0165, 0.0625,
       0.0225, 0.003 , 0.163 , 0.0085, 0.028 , 0.046 ]), array([0.    , 0.    , 0.    , 0.    , 0.    , 0.007 , 0.0075, 0.061 ,
       0.016 , 0.0005, 0.166 , 0.0135, 0.0315, 0.036 ])]


array([0.    , 0.    , 0.    , 0.    , 0.    , 0.0068, 0.0101, 0.063 ,
       0.017 , 0.0024, 0.1649, 0.0108, 0.0311, 0.0406])

In [22]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
  
actual = y_test
predicted = pred 
results = confusion_matrix(actual, predicted) 
  
print ('Confusion Matrix :')
print (results) 
print ('Accuracy Score :',accuracy_score(actual, predicted))
print ('Report : ',classification_report(actual, predicted))

NameError: name 'pred' is not defined

In [45]:
nof_nans = numpy.sum(numpy.isnan(X_test))
print('fraction of nans in X: {:.3f}'.format(nof_nans/numpy.prod(X_train.shape)))
    
prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True)
prf_cls.fit(X=X_train, y=y_train)
print('PRF: {}'.format(prf_cls.score(X=X_test, y=y_test)))

imp = Imputer(missing_values=numpy.nan, strategy='median')
X_train_w_nans_imp = imp.fit_transform(X_train)
X_test_w_nans_imp = imp.fit_transform(X_test)
dX_train_w_nans_imp = imp.fit_transform(dX_train)
dX_test_w_nans_imp = imp.fit_transform(dX_test)

RF = RandomForestClassifier(n_estimators=n_trees,n_jobs=-1, bootstrap=True)
RF.fit(X_train_w_nans_imp, y_train)
print('RF: {}'.format(RF.score(X_test_w_nans_imp,y_test)))

prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True)
prf_cls.fit(X=X_train_w_nans_imp, y=y_train, dX=dX_train_w_nans_imp)
print('PRF: {}'.format(prf_cls.score(X=X_test_w_nans_imp, y=y_test, dX=dX_test_w_nans_imp)))

fraction of nans in X: 0.033
PRF: 0.753
RF: 0.748
PRF: 0.735


In [42]:
data = data[data[' cflvsc.MainVarType'] != 'Spurious']

for inverse_column in [' cflvsc.HeightKfi2',' cflvsc.HeightLfl2',' cflvsc.HeightLSG',' cflvsc.HeightPDM',
        ' cflvsc.HeightSTR']:
    columnName = inverse_column.replace(" cflvsc.",'Inverse')
    data[columnName] = numpy.ones(len(data[inverse_column]))
    data[columnName] = data[columnName].div(data[inverse_column])

#Maybe flag objects with low Ncorrelation with a different MainVarType flag - that was it can identify them as bad

X=data[[' cflvsc.zAperMag3',' cflvsc.yAperMag3',' cflvsc.jAperMag3', ' cflvsc.hAperMag3',
        ' cflvsc.ksAperMag3',
       ' cflvsc.FreqKfi2',' cflvsc.FreqLfl2',' cflvsc.FreqLSG',' cflvsc.FreqPDM',
       ' cflvsc.FreqSTR',
       ' cflvsc.Avar',' cflvsc.Kfi2', ' cflvsc.L2',' cflvsc.ebv',' cflvsc.Ngoodmeasures',
       ]].to_numpy()

dX = data[[' cflvsc.zAperMag3Err', ' cflvsc.yAperMag3Err',' cflvsc.jAperMag3Err',' cflvsc.hAperMag3Err', 
           ' cflvsc.ksAperMag3Errs',
        'InverseHeightKfi2','InverseHeightLfl2','InverseHeightLSG','InverseHeightPDM',
        'InverseHeightSTR',
           'zeroes','zeroes','zeroes',' cflvsc.ebverr','zeroes'
          ]].to_numpy()

y=data[' cflvsc.MainVarType'].to_numpy()
print(X)
print(y)

n_objects = X.shape[0]
n_features = X.shape[1]
print(n_objects, 'objects,', n_features, 'features')

shuffled_inds = numpy.random.choice(numpy.arange(n_objects),n_objects,replace=False)
shuffled_inds = numpy.random.choice(shuffled_inds,len(shuffled_inds),replace=False)
print("Re-Shuffled Indices? ", shuffled_inds)
n_train = 10000
n_test = 1000
print('Train set size = {}, Test set size = {}'.format(n_train, n_test))

nf = n_features
train_inds = shuffled_inds[:n_train]
X_train = X[train_inds][:,:nf]
y_train = y[train_inds]
dX_train = dX[train_inds][:,:nf]

test_inds = shuffled_inds[n_train:(n_train + n_test)]
X_test = X[test_inds][:,:nf]
y_test = y[test_inds]
dX_test = dX[test_inds][:,:nf]

[[14.76163  14.657574 14.700644 ...  1.425342  0.       82.      ]
 [      nan       nan       nan ...  5.70632   0.       84.      ]
 [      nan       nan       nan ...  1.16128   0.       84.      ]
 ...
 [      nan       nan       nan ...  0.561949  1.122712 74.      ]
 [      nan       nan       nan ... 16.451157  1.941331 65.      ]
 [      nan       nan       nan ... 44.133835  1.918944 58.      ]]
['  RR' '  RR' '  RR' ... '  E' '  EB' '  EA']
174359 objects, 15 features
Re-Shuffled Indices?  [114451 162681   4658 ... 148918 140746  19042]
Train set size = 10000, Test set size = 1000


In [43]:
n_trees = 100
prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True)
prf_cls.fit(X=X_train, y=y_train, dX = dX_train)
print('PRF: {}'.format(prf_cls.score(X_test, y=y_test, dX=dX_test)))

PRF: 0.739


# Missing values
* the original data does not have missing values. We add missing values to the data by setting in random elements in X to numpy.nan
* we compare the results to the original RF where the missing values are imputed

In [ ]:
from sklearn.impute import SimpleImputer as Imputer # for new versions for sklearn
#from sklearn.preprocessing import Imputer # old versions of sklearn

import matplotlib.pyplot as plt

def insert_nans(X_train, X_test, nan_frac):
    X_train_w_nans = X_train.copy()
    X_test_w_nans = X_test.copy()

    nof_nans = int( numpy.prod(X_train.shape) * nan_frac )
    for i in range(nof_nans):
        o = numpy.random.choice(n_train)
        f = numpy.random.choice(nf)
        X_train_w_nans[o,f] = numpy.nan

    nof_nans = int( numpy.prod(X_test.shape) * nan_frac )
    for i in range(nof_nans):
        o = numpy.random.choice(n_test)
        f = numpy.random.choice(nf)
        X_test_w_nans[o,f] = numpy.nan
    
    imp = Imputer(missing_values=numpy.nan, strategy='median')
    X_train_w_nans_imp = imp.fit_transform(X_train_w_nans)
    X_test_w_nans_imp = imp.fit_transform(X_test_w_nans)
    
    return X_train_w_nans, X_test_w_nans, X_train_w_nans_imp, X_test_w_nans_imp

def prf_rf_nans_compare_single(X_train, X_test, n_trees, nan_frac):
    
    X_train_w_nans, X_test_w_nans, X_train_w_nans_imp, X_test_w_nans_imp = insert_nans(X_train, X_test, nan_frac)

    nof_nans = numpy.sum(numpy.isnan(X_train_w_nans))
    print('fraction of nans in X: {:.3f}'.format(nof_nans/numpy.prod(X_train.shape)))
    
    print('Accuracy for {} trees --- '.format(n_trees))

    prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True)
    prf_cls.fit(X=X_train_w_nans, y=y_train)
    print('PRF: {}'.format(prf_cls.score(X=X_test_w_nans, y=y_test)))

    RF = RandomForestClassifier(n_estimators=n_trees,n_jobs=-1, bootstrap=True)
    RF.fit(X_train, y_train)
    print('RF: {}'.format(RF.score(X_test_w_nans_imp,y_test)))
    
    return

def plot_prf_rf_cmpr(nan_frac_vec_true, prf_scores, prf_scores_stds, rf_scores, rf_scores_stds):
    plt.figure(figsize = (10,7))
    lw = 7
    alpha = 0.5
    alpha_eb = 0.3
    ms = 25
    
    markers, caps, bars =plt.errorbar(x=nan_frac_vec_true,y=prf_scores,yerr=prf_scores_stds,capsize=10, label = 'PRF',fmt ='--*', markersize= 15, capthick=3)
    [bar.set_alpha(alpha_eb) for bar in bars]
    markers, caps, bars =plt.errorbar(x=nan_frac_vec_true,y=rf_scores,yerr=rf_scores_stds,capsize=10,label = 'RF',fmt ='--*', markersize= 15, capthick=3)
    [bar.set_alpha(alpha_eb) for bar in bars]

    plt.legend(fontsize = 20)
    plt.xlabel('Fraction of NaNs in X', fontsize = 20)
    plt.ylabel('Accuracy', fontsize = 20)
    plt.xticks(fontsize = 20)
    yticks = plt.gca().get_yticks()
    yticks_p = ['{}%'.format('%.1f' % (yt*100)) for yt in yticks]
    plt.yticks(yticks, yticks_p, fontsize = 20)
    plt.tight_layout()
    plt.show()
    
    return

def prf_rf_nans_compare_full(X_train, X_test, n_itr, n_trees):

    nof_elements = numpy.prod(X_train.shape)
    
    prf_scores = []
    prf_scores_stds = []
    rf_scores = []
    rf_scores_stds = []

    nan_frac_vec = [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.1, 1.5, 2]
    nan_frac_vec_true = numpy.zeros(len(nan_frac_vec))
    for n_idx, nan_frac in enumerate(nan_frac_vec):

        X_train_w_nans, X_test_w_nans, X_train_w_nans_imp, X_test_w_nans_imp = insert_nans(X_train, X_test, nan_frac)

        scores = numpy.zeros(n_itr)
        for i in range(n_itr): 
            prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True, keep_proba=0.01)
            prf_cls.fit(X=X_train_w_nans, y=y_train)
            scores[i] = prf_cls.score(X=X_test_w_nans, y=y_test)
        prf_scores += [scores.mean()]
        prf_scores_stds += [scores.std()]

        scores = numpy.zeros(n_itr)
        for i in range(n_itr): 
            RF = RandomForestClassifier(n_estimators=n_trees,n_jobs=-1, bootstrap=True)
            RF.fit(X_train_w_nans_imp, y_train)
            scores[i] = RF.score(X_test_w_nans_imp,y_test)
        rf_scores += [scores.mean()]
        rf_scores_stds += [scores.std()]
        
        nof_nans = numpy.sum(numpy.isnan(X_train_w_nans))
        nan_frac_vec_true[n_idx] = nof_nans/nof_elements

        print('nan fraction:{:.2f}, PRF:{:.3f}, RF:{:.3f}'.format(nan_frac_vec_true[n_idx],prf_scores[-1], rf_scores[-1]))
        

    
    plot_prf_rf_cmpr(nan_frac_vec_true, prf_scores, prf_scores_stds, rf_scores, rf_scores_stds)
    
    return nan_frac_vec_true, prf_scores, rf_scores, prf_scores_stds, rf_scores_stds


def prf_rf_nans_test_set_only_compare_full(X_train, X_test, n_itr, n_trees):

    nof_elements = numpy.prod(X_train.shape)
    
    prf_scores = []
    prf_scores_stds = []
    rf_scores = []
    rf_scores_stds = []

    nan_frac_vec = [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.1, 1.5, 2]
    nan_frac_vec_true = numpy.zeros(len(nan_frac_vec))
    for n_idx, nan_frac in enumerate(nan_frac_vec):

        X_train_w_nans, X_test_w_nans, X_train_w_nans_imp, X_test_w_nans_imp = insert_nans(X_train, X_test, nan_frac)

        scores = numpy.zeros(n_itr)
        for i in range(n_itr): 
            prf_cls = PRF.prf(n_estimators=n_trees,  bootstrap=True, keep_proba=0.01)
            prf_cls.fit(X=X_train, y=y_train)
            scores[i] = prf_cls.score(X=X_test_w_nans, y=y_test)
        prf_scores += [scores.mean()]
        prf_scores_stds += [scores.std()]

        scores = numpy.zeros(n_itr)
        for i in range(n_itr): 
            RF = RandomForestClassifier(n_estimators=n_trees,n_jobs=-1, bootstrap=True)
            RF.fit(X_train, y_train)
            scores[i] = RF.score(X_test_w_nans_imp,y_test)
        rf_scores += [scores.mean()]
        rf_scores_stds += [scores.std()]
        
        nof_nans = numpy.sum(numpy.isnan(X_train_w_nans))
        nan_frac_vec_true[n_idx] = nof_nans/nof_elements

        print('nan fraction:{:.2f}, PRF:{:.3f}, RF:{:.3f}'.format(nan_frac_vec_true[n_idx],prf_scores[-1], rf_scores[-1]))
        

    plot_prf_rf_cmpr(nan_frac_vec_true, prf_scores, prf_scores_stds, rf_scores, rf_scores_stds)
    
    return nan_frac_vec_true, prf_scores, rf_scores, prf_scores_stds, rf_scores_stds

In [ ]:
_ = prf_rf_nans_compare_single(X_train=X_train, X_test=X_test, n_trees=1, nan_frac=0.5)

In [ ]:
_ = prf_rf_nans_compare_single(X_train=X_train, X_test=X_test, n_trees=10, nan_frac=0.5)

In [ ]:
_ = prf_rf_nans_compare_single(X_train=X_train, X_test=X_test, n_trees=100, nan_frac=0.5)

# Missing values in both train and test sets
* The PRF accuracy is higher for a single tree, but the same as a regular RF for a large number of trees

In [ ]:
_ = prf_rf_nans_compare_full(X_train=X_train, X_test=X_test, n_itr=10, n_trees=1)

In [ ]:
_ = prf_rf_nans_compare_full(X_train=X_train, X_test=X_test, n_itr = 1, n_trees = 25)

# Missing values in test set only
* The PRF accuracy is higher even for a large number of trees

In [ ]:
_ = prf_rf_nans_test_set_only_compare_full(X_train=X_train, X_test=X_test, n_itr = 10, n_trees = 1)

In [ ]:
_ = prf_rf_nans_test_set_only_compare_full(X_train=X_train, X_test=X_test, n_itr = 5, n_trees = 10)

In [ ]:
_ = prf_rf_nans_test_set_only_compare_full(X_train=X_train, X_test=X_test, n_itr = 1, n_trees = 25)